In [1]:
#Import Stuff 
import os
import cv2
import math
import numpy as np
from skimage.util import random_noise
import SimpleITK as sitk

In [2]:
# Extract Data from Amira Files
for i in range(51,59):
    print("i:",i)
    # Open two empty dictionaries for later use
    files = {}
    landmarks = {}

    for filename in os.listdir():
        if os.path.isfile(filename) and filename.startswith("S0{}_Landmarks".format(i)) and filename.endswith(".landmarkAscii") and not filename in files: 
    #os.path.isfile() method in Python is used to check whether the specified path is an existing regular file or not.
    #the if condition checks for a regular file and if ends with particular extension and also checks if the particular file is already in the dictionary
            with open(filename, "r") as file:
                #opens the particular file in read mode
                files[filename] = file.readlines()
                #file gets added to dictionary 'files'
                # reads all the files into a dictionary
                #readlines() reads line by line as a list

    #prints all the files with given extension in the system
    for filename, text in files.items(): 
        print(filename)# prints the list of files with the given extension
        #print(text)# prints the entire contents of the text file 

    ## Extracting Coordinates

    # create a set of lists for coordinates
    x_coordinates = [] #x-coordinate
    y_coordinates = [] #y-coordinate   
    coordinates = [] #x,y-coordinate put together
    linecount = 0 #utility variable

    #extraction of coordinates
    for filename, text in files.items():
        #files is the dict; filename is teh keyword and text is teh value
        # first for loop to enter set of files
        for texts in text:
            # second for loop to enter inside inside the file and access its contents
            if texts.startswith('@1'):
                #looks for line that startswith @1 from the coordinates are listed 
                 linecount = 1
                 name = filename
                 #print(texts)
                 #print(filename)
                 continue
            if linecount == 1 and texts.isspace() == False and filename==name: 
                #looks for the if @1 has been encountered and counts till an empty space
                #print(texts)
                 x = list(map(float,texts.split()))[0]#convert the values into float
                 y = list(map(float,texts.split()))[1]
                 x_coordinates.append(x)
                 y_coordinates.append(y)

    #coordinates.extend(list(zip(x_coordinates,y_coordinates)))
    print("Landmarks Acquired!")
    #print(len(coordinates))

    #fixing the resolution

    x_coordinates = [ x/.868 for x in x_coordinates] # division to get the resolution right
    y_coordinates = [ y/.868 for y in y_coordinates]

    x_coordinates = [int(i) for i in x_coordinates]
    y_coordinates = [int(i) for i in y_coordinates]

    x = x_coordinates
    y = y_coordinates

    # The final Coordinates are obtained as a tuple
    coordinates = list(zip(x_coordinates, y_coordinates))
    #print(coordinates)
    print("Number of landmarks:",len(coordinates))

    # Access the coordinates and crop the image around the coordinates

    #Read the image
    source_path='//ibs9010/current_data/Data_Keshav/masked_for_rabies_auto_detection/MG48_3day_bs/Image/pia_only/S{}_Pia_Only.tif'.format(i)
    image=cv2.imread(source_path)
    
    # Image Basics
    print(image.shape)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(image.shape)
    #im = sitk.ReadImage(source_path)
    #image = sitk.GetArrayFromImage(im)
    
    import copy
    bkgd_image = copy.copy(image)#image.copy()

    h = image.shape[0]
    w = image.shape[1]
    print("Height, Width = ",image.shape)
    mini_centre_image = []
    bkgd_image_list = []
    mini_partial_image = []
    s = 7
    
    def crop():
        for a,b in coordinates:#coordinates is teh list of landmarks obtained from .ascii files
            #print("a,b:",a,b)
            x1 = a-35
            x2 = a+35
            y1 = b-35
            y2 = b+35
            #print('Centre crop')
            im1 = image[y1:y2, x1:x2]

            dark_patch = np.zeros_like(im1)
            noise_img = random_noise(dark_patch, mode='gaussian',mean=0.02,var=0.01)
            noise_img = np.array(255*noise_img, dtype = 'uint8')
            #print((noise_img))

            #gauss = np.random.normal(0,0.3,im1.size)
            #gauss = gauss.reshape(im1.shape[0],im1.shape[1]).astype('uint8')

            bkgd_image[y1:y2, x1:x2] = noise_img
            mini_centre_image.append(im1)

            x3 = x1 + s
            y3 = y1 + s
            x4 = x2 + s
            y4 = y2 + s
            im2 = image[y3:y4, x3:x4]
            mini_partial_image.append(im2)

            x5 = x1 + 2*s
            y5 = y1 + 2*s
            x6 = x2 + 2*s
            y6 = y2 + 2*s
            im3 = image[y5:y6, x5:x6]
            mini_partial_image.append(im3)

            x7 = x1 + 3*s
            y7 = y1 + 3*s
            x8 = x2 + 3*s
            y8 = y2 + 3*s
            im4 = image[y7:y8, x7:x8]
            mini_partial_image.append(im4)

            x9 = x1 + 4*s
            y9 = y1 + 4*s
            x10 = x2 + 4*s
            y10 = y2 + 4*s
            im5 = image[y9:y10, x9:x10]
            mini_partial_image.append(im5)


    crop()            
    print("Saving",len(mini_centre_image),"centre images.")
    print("Saving",len(mini_partial_image),"right shifted partial images.")



    Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/test/Neuron/'
    #Save the strided images
    for k in range(len(mini_centre_image)):
        cv2.imwrite(os.path.join(Dest_Path,"S0{}_centre_".format(i)+str(k)+".tif"), mini_centre_image[k])
    for k in range(len(mini_partial_image)):
        cv2.imwrite(os.path.join(Dest_Path,"S0{}_right_".format(i)+str(k)+".tif"), mini_partial_image[k])

    Bkgd_Dest_Path = '//ibs9010/current_data/Data_Keshav/masked_for_rabies_auto_detection/MG48_3day_bs/BackgroundImages/'
    cv2.imwrite(os.path.join(Bkgd_Dest_Path,"Bkgd_S0{}.tif".format(i)), bkgd_image)

i: 51
S051_Landmarks.landmarkAscii
Landmarks Acquired!
Number of landmarks: 190
(9536, 10340, 3)
(9536, 10340)
Height, Width =  (9536, 10340)
Saving 190 centre images.
Saving 760 right shifted partial images.
i: 52
S052_Landmarks.landmarkAscii
Landmarks Acquired!
Number of landmarks: 129
(9577, 10063, 3)
(9577, 10063)
Height, Width =  (9577, 10063)
Saving 129 centre images.
Saving 516 right shifted partial images.
i: 53
S053_Landmarks.landmarkAscii
Landmarks Acquired!
Number of landmarks: 160
(9528, 10352, 3)
(9528, 10352)
Height, Width =  (9528, 10352)
Saving 160 centre images.
Saving 640 right shifted partial images.
i: 54
S054_Landmarks.landmarkAscii
Landmarks Acquired!
Number of landmarks: 142
(9519, 10297, 3)
(9519, 10297)
Height, Width =  (9519, 10297)
Saving 142 centre images.
Saving 568 right shifted partial images.
i: 55
S055_Landmarks.landmarkAscii
Landmarks Acquired!
Number of landmarks: 183
(9577, 11125, 3)
(9577, 11125)
Height, Width =  (9577, 11125)
Saving 183 centre imag

In [ ]:
for m in range(51,59):
    print("m:",m) 
    #Read the image
    #source_path = '/Users/keshavaprasad/Desktop/image/MG48_3day_bs/testingcrop/S051_00.tif'
    #source_path = '/Users/keshavaprasad/Desktop/image/S25/S25_00.tif'
    source_path2 = '//ibs9010/current_data/Data_Keshav/masked_for_rabies_auto_detection/MG48_3day_bs/BackgroundImages/Bkgd_S0{}.tif'.format(m)
    Bimage=cv2.imread(source_path2)

    # Image Basics
    print(Bimage.shape)
    # Get Shape
    p = Bimage.shape[0]# p
    q = Bimage.shape[1]#q
    print("Height:",p,"Width:",q)

    # Defining the dimensions of a tile 
    #a = b = 50
    a = b = 70
    #************************
    bkgd_cropped_tile_images = [] #list for storing all cropped tile images

    # Cropping image Sequentially
    i = j = cropped_tile_counter = correct_tile_counter = wrong_dim_tile_counter = correct_dim_tile_counter= 0

    # Carry oout the sequential cropping store data into respective lists
    for i in range(0,math.floor(p),a):#range(start, stop, step) # to get the resolution right!
        for j in range(0,math.floor(q),b):
            #Defining the cropping box
            #tile = image[j:j+b,i:i+a]#y1: y2, x1: x2
            y1 = j
            x1 = i
            y2 = j+b
            x2 = i+a

            tile = Bimage[x1:x2,y1:y2]# making crops of each subimage and then storing in tile variable and then appended to cropped_tiles list
            #vertex = [x1,x2,y1,y2] # list of vertex values of each tiles
            cropped_tile_counter +=1

            # Calculating mean for each tile and
            #mean = np.mean(tile)

            # checking for tiles with wrong dimensions
            #if tile.shape[0]!=50 or tile.shape[1]!= 50:
            if tile.shape[0]!=70 or tile.shape[1]!= 70:
                wrong_dim_tile_counter += 1

            # Keeping only tiles with correct dimensions # Append the vertex for each of these tiles into vertices list 
            # Append the correct_tile_counter variable into ctc list

            #if tile.shape[0] == 50 and tile.shape[1] == 50:
            if tile.shape[0] == 70 and tile.shape[1] == 70 and not np.all((np.array(tile) <5)):
                correct_dim_tile_counter +=1
                correct_tile_counter +=1
                #cropped_tiles.append(tile)
                bkgd_cropped_tile_images.append(tile)
                #ctc.append([cropped_tile_counter,correct_tile_counter])
                #ctc.append(correct_tile_counter)
                #print("correct_tile_counter:",correct_tile_counter,"with vertex:",vertex)
                #vertices.append(vertex)                        
    ### THE TWO KEY TAKEAWAYS:number of croppedtiles and respective vertex of each cropped tile

    Bkgd_Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/test/Background/'

    # Delete all exiting images
    #import glob
    #os.chdir(Bkgd_Dest_Path)
    #files=glob.glob('*.tif')
    #for filename in files:
    #    os.unlink(filename)
    # Saving the tiles in a respective folder    
    for k in range(1,len(bkgd_cropped_tile_images)):
            #pass
            #print("K:",k)
            #cv2.imwrite(str(k)+ ".tif", tiles[k])
            cv2.imwrite(os.path.join(Bkgd_Dest_Path,"S{}_bkgd_".format(m)+str(k)+".tif"), bkgd_cropped_tile_images[k])

m: 51
(9536, 10340, 3)
Height: 9536 Width: 10340
m: 52
(9577, 10063, 3)
Height: 9577 Width: 10063
m: 53
(9528, 10352, 3)
Height: 9528 Width: 10352
m: 54
(9519, 10297, 3)
Height: 9519 Width: 10297
m: 55
(9577, 11125, 3)
Height: 9577 Width: 11125
